# Etapa 2 - Avaliação de Modelo Personalized PageRank

In [1]:
from models.personalized_pagerank import PersonalizedPageRank

import pickle
import dgl

In [2]:
with open("../dados-processados/dgl_network.pickle", 'rb') as _file:
    dgl_network = pickle.load(_file)

In [3]:
model = PersonalizedPageRank( 
    visit_target    = 500,
    reset_prob      = 0.0,
    dgl_network     = dgl_network
)

In [4]:
with open("../dados-processados/dataset_challenge.pickle", 'rb') as _file:
    testset = pickle.load(_file)

In [5]:
model.network.edata['weights'] = model.network.edata['weights'].double()

In [6]:
from tqdm import tqdm


preds = []

for pid, name, playlist in tqdm(testset[:2000]):
    if len(playlist) == 0:
        prediction = model.predict([[0]])
    else:
        prediction = model.predict([playlist])
    
    preds.append( (pid, prediction[0]) )

  0%|          | 6/2000 [00:01<07:55,  4.19it/s]

501
501
501
501
501
501
501
501
501
501


  1%|          | 17/2000 [00:02<02:17, 14.47it/s]

501
501
501
501
501
501
501
501
501
501
501


  1%|▏         | 28/2000 [00:02<01:15, 25.95it/s]

...

101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101


100%|█████████▉| 1993/2000 [00:46<00:00, 45.57it/s]

101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101


100%|██████████| 2000/2000 [00:46<00:00, 42.87it/s]

101
101
101
101
101
101
101
101
101
101
101
101


In [7]:
import pandas as pd


preds_df = pd.DataFrame(preds, columns=['pid','result'])

preds_df

,pid,result
0,1000002,"[tensor(3416), tensor(207356), tensor(8308), t..."
1,1000003,"[tensor(14771), tensor(6946), tensor(14482), t..."
2,1000004,"[tensor(5543), tensor(7070), tensor(4271), ten..."
3,1000006,"[tensor(4102), tensor(4742), tensor(14077), te..."
4,1000007,"[tensor(4625), tensor(1820), tensor(14857), te..."
...,...,...
1995,1003466,"[tensor(9774), tensor(49062), tensor(50692), t..."
1996,1003467,"[tensor(350), tensor(73470), tensor(48962), te..."
1997,1003469,"[tensor(13783), tensor(3048), tensor(11682), t..."
1998,1003471,"[tensor(7526), tensor(12923), tensor(13181), t..."


In [8]:
preds_df['tam'] = preds_df.result.apply(lambda x: len(x))

preds_df

,pid,result,tam
0,1000002,"[tensor(3416), tensor(207356), tensor(8308), t...",500
1,1000003,"[tensor(14771), tensor(6946), tensor(14482), t...",500
2,1000004,"[tensor(5543), tensor(7070), tensor(4271), ten...",500
3,1000006,"[tensor(4102), tensor(4742), tensor(14077), te...",500
4,1000007,"[tensor(4625), tensor(1820), tensor(14857), te...",500
...,...,...,...
1995,1003466,"[tensor(9774), tensor(49062), tensor(50692), t...",499
1996,1003467,"[tensor(350), tensor(73470), tensor(48962), te...",499
1997,1003469,"[tensor(13783), tensor(3048), tensor(11682), t...",500
1998,1003471,"[tensor(7526), tensor(12923), tensor(13181), t...",499


# The first non-commented/blank line should start with "team_info" 
# and then include team name, and contact email address. 
#
# for each challenge playlist there should be a line of the form:
#     pid, trackuri_1, trackuri_2, track_uri_3, ..., track_uri_499, track_uri_500
#
# before submission, the csv should be gzipped.

In [9]:
preds_df.describe()

,pid,tam
count,2.000000e+03,2000.000000
mean,1.001764e+06,480.279500
std,1.001408e+03,70.345665
min,1.000000e+06,22.000000
25%,1.000914e+06,499.000000
50%,1.001756e+06,500.000000
75%,1.002627e+06,500.000000
max,1.003476e+06,500.000000
